# Data Ingestion Module Verification

This notebook tests the `src.ingestion` module which handles:
- **Loading mtRAG data** from JSONL files
- **Parent-Child Chunking** for optimal retrieval (large chunks for context, small for search)
- **Vector Store Creation** using Qdrant + BGE-M3 embeddings

Uses a **small subset (50 docs)** for fast testing.

In [1]:
import sys
import os
import json
import zipfile

sys.path.append(os.path.abspath(".."))
PROJECT_ROOT = os.path.abspath("..")
QDRANT_PATH = os.path.join(PROJECT_ROOT, "qdrant_ingestion_test")
MAX_DOCS = 50

print(f"Project root: {PROJECT_ROOT}")
print(f"Test subset size: {MAX_DOCS} documents")

Project root: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8
Test subset size: 50 documents


## Step 0: Prepare Test Data
Extract the corpus and create a small subset for fast testing.

In [2]:
# Extract corpus if needed
corpus_dir = os.path.join(PROJECT_ROOT, "dataset/corpora/passage_level")
jsonl_file = os.path.join(corpus_dir, "govt.jsonl")
zip_file = os.path.join(corpus_dir, "govt.jsonl.zip")

if not os.path.exists(jsonl_file) and os.path.exists(zip_file):
    print("📦 Extracting corpus...")
    with zipfile.ZipFile(zip_file, 'r') as zf:
        zf.extractall(corpus_dir)
    print("✅ Corpus extracted")
else:
    print(f"✅ Corpus ready: govt.jsonl")

# Create test subset
test_file = os.path.join(PROJECT_ROOT, "data/test_subset.jsonl")
os.makedirs(os.path.dirname(test_file), exist_ok=True)

print(f"📝 Creating test subset with {MAX_DOCS} documents...")
with open(jsonl_file, 'r') as f_in, open(test_file, 'w') as f_out:
    for i, line in enumerate(f_in):
        if i >= MAX_DOCS:
            break
        f_out.write(line)
print(f"✅ Test file created: {test_file}")

✅ Corpus ready: govt.jsonl
📝 Creating test subset with 50 documents...
✅ Test file created: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/test_subset.jsonl


## Step 1: Test `load_and_chunk_data()`

This function:
1. Loads documents from JSONL
2. Applies **Parent-Child Chunking**:
   - Parent chunks: 1200 chars (full context for LLM)
   - Child chunks: 400 chars (indexed for search)
3. Stores parent text in child metadata for retrieval

In [3]:
from src.ingestion import load_and_chunk_data

print("🔄 Loading and chunking data...")
docs = load_and_chunk_data(test_file)

print(f"\n📊 Results:")
print(f"   • Total chunks created: {len(docs)}")
print(f"   • Avg chunks per document: {len(docs) / MAX_DOCS:.1f}")

print(f"\n📄 Sample chunk:")
sample = docs[0]
print(f"   • Child content (indexed): {sample.page_content[:100]}...")
print(f"   • Parent text length: {len(sample.metadata.get('parent_text', ''))} chars")
print(f"   • Metadata keys: {list(sample.metadata.keys())}")

print("\n✅ load_and_chunk_data() working correctly!")

/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔄 Loading and chunking data...
--- LOADING DATA FROM /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/test_subset.jsonl ---
Loaded 50 documents.
--- STARTING PARENT-CHILD SPLITTING ---

📊 Results:
   • Total chunks created: 409
   • Avg chunks per document: 8.2

📄 Sample chunk:
   • Child content (indexed): FAQs • What type of seats can I book?
FAQs • What type of seats can I book?

 

Skip to Main Content...
   • Parent text length: 776 chars
   • Metadata keys: ['doc_id', 'title', 'url', 'source', 'parent_text', 'parent_title', 'parent_url', 'parent_id']

✅ load_and_chunk_data() working correctly!


## Step 2: Test `build_vector_store()`

This function:
1. Creates HuggingFace embeddings (BGE-M3)
2. Initializes Qdrant local database
3. Indexes all chunks with their embeddings

In [4]:
from src.ingestion import build_vector_store

# Use only first 30 chunks for speed
docs_subset = docs[:30]
print(f"🔄 Building vector store with {len(docs_subset)} chunks...")
print("   (Using subset for faster testing)")

vectorstore = build_vector_store(docs_subset, persist_dir=QDRANT_PATH)

print("\n✅ build_vector_store() working correctly!")

🔄 Building vector store with 30 chunks...
   (Using subset for faster testing)
--- BUILDING VECTOR STORE ---


/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/src/ingestion.py:108: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


--- VECTOR STORE BUILT AND SAVED ---

✅ build_vector_store() working correctly!


## Step 3: Verify Qdrant Collection

In [5]:
info = vectorstore.client.get_collection("mtrag_collection")

print(f"📊 Collection Statistics:")
print(f"   • Points (vectors): {info.points_count}")
print(f"   • Status: {info.status}")

print("\n✅ Collection created and verified!")

📊 Collection Statistics:
   • Points (vectors): 30
   • Status: green

✅ Collection created and verified!


## Step 4: Test Similarity Search

Verify that the vector store can find relevant documents.

In [6]:
query = "government regulations"
print(f"🔍 Testing search with query: '{query}'")

results = vectorstore.similarity_search_with_score(query, k=3)

print(f"\n📊 Found {len(results)} results:")
for i, (doc, score) in enumerate(results):
    print(f"\n   Result {i+1} (similarity: {score:.4f}):")
    print(f"   • Child chunk: {doc.page_content[:80]}...")
    parent = doc.metadata.get('parent_text', '')
    print(f"   • Parent context: {parent[:80]}..." if parent else "   • No parent")

print("\n✅ Similarity search working correctly!")

🔍 Testing search with query: 'government regulations'

📊 Found 3 results:

   Result 1 (similarity: 0.4917):
   • Child chunk: Adopted City Council Ordinances

Adopted City Council Resolutions

 

 
HomeFAQs...
   • Parent context: FAQs • What type of seats can I book?
FAQs • What type of seats can I book?

 

...

   Result 2 (similarity: 0.4531):
   • Child chunk: FAQs • What type of seats can I book?
FAQs • What type of seats can I book?

 

...
   • Parent context: FAQs • What type of seats can I book?
FAQs • What type of seats can I book?

 

...

   Result 3 (similarity: 0.4520):
   • Child chunk: 3.
What type of seats can I book?...
   • Parent context: 2.
Where can I board MoGo?

Pick-up and drop-off is available at designated virt...

✅ Similarity search working correctly!


## Cleanup
Remove test files after verification.

In [7]:
import shutil

# Close client first
vectorstore.client.close()

# Remove test files
if os.path.exists(QDRANT_PATH):
    shutil.rmtree(QDRANT_PATH)
    print(f"🗑️ Removed test database: {QDRANT_PATH}")

if os.path.exists(test_file):
    os.remove(test_file)
    print(f"🗑️ Removed test subset: {test_file}")

print("\n✅ All ingestion tests passed!")

🗑️ Removed test database: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/qdrant_ingestion_test
🗑️ Removed test subset: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/test_subset.jsonl

✅ All ingestion tests passed!
